In [ ]:
from tensorboard.backend.event_processing import event_accumulator
import tensorflow as tf

import pandas as pd

import os
import re

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# CIFAR10 standard (i.e., in distribution)

In [ ]:
base_path = "BASE_DATA_PATH/subset_vi/cifar10_moreseeds"

In [ ]:
def code_to_name(code):
    if code == "1":
        return "Stoc Input"
    elif code == "0":
        return "Deterministic"
    elif code == "8192":
        return "Stoc Output"
    elif code == "12288":
        return "Last 2 Stoc"
    elif code == "16383":
        return "All Stoc"

In [ ]:
def path_to_results(path, stats=None):
    if not stats:
        stats = ["test/ece", "test/accuracy", "test/negative_log_likelihood"]
    ea = event_accumulator.EventAccumulator(
        path,
        size_guidance={'tensors': 0}
    )
    ea.Reload()
    
    results = {}
    
    for stat in stats:
        try:
            _, _, val = ea.Tensors(stat)[-1]
            val = tf.make_ndarray(val)
            results[stat] = float(val)
        except:
            print(f"{stat} missing from {path}")
    
    return results

In [ ]:
# get all events
all_results_list = []

for subdir, dirs, files in os.walk(base_path):
    for f in files:
        if ".tfevent" in f:
            code_seed = re.search("[0-9]+_[0-9]+", subdir).group()
            code, seed = code_seed.split('_')
            
            events_file = os.path.join(subdir, f)
            results = path_to_results(events_file)
            all_results_list.append({**{
                "code": code,
                "seed": seed,
                "model_name": code_to_name(code),
            }, **results})

all_results_df = pd.DataFrame(all_results_list)

In [ ]:
all_results_df["test/ece"] = 100*all_results_df["test/ece"]
all_results_df["test/accuracy"] = 100*all_results_df["test/accuracy"]

In [ ]:
all_results_df.to_csv("cifar10_all_results_df.csv")

# CIFAR100 Standard

In [ ]:
base_path = "BASE_DATA_PATH/subset_vi/cifar100_moreseeds"

In [ ]:
# get all events
all_results_list = []

for subdir, dirs, files in os.walk(base_path):
    for f in files:
        if ".tfevent" in f:
            code_seed = re.search("[0-9]+_[0-9]+", subdir).group()
            code, seed = code_seed.split('_')
            
            events_file = os.path.join(subdir, f)
            results = path_to_results(events_file)
            all_results_list.append({**{
                "code": code,
                "seed": seed,
                "model_name": code_to_name(code),
            }, **results})

all_results_df = pd.DataFrame(all_results_list)

In [ ]:
all_results_df["test/ece"] = 100*all_results_df["test/ece"]
all_results_df["test/accuracy"] = 100*all_results_df["test/accuracy"]

In [ ]:
all_results_df.to_csv("cifar100_all_results_df.csv")